In [1]:
from __future__ import print_function
import os
import sys
import math
import pickle
import boto3
import os
import numpy as np
import kg
import encoding
import pandas as pd
# from tqdm import tqdm
import time
import argparse
import logging
import re
# tqdm.pandas()
# pandarallel.initialize(progress_bar=True)
# bucket = os.environ.get("BUCKET_NAME", " ")
# raw_data_folder = os.environ.get("RAW_DATA", " ")
# logger = logging.getLogger()
# logger.setLevel(logging.INFO)
# tqdm_notebook().pandas()
s3client = boto3.client('s3')

Using backend: pytorch


sys path is ['/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python36.zip', '/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6', '/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/lib-dynload', '', '/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages', '/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/dglke-0.1.0.dev0-py3.6.egg', '/home/ec2-user/workplace/Aws-gcr-sc-recommender-system/src/offline/docker/batch/container/batch/src/fasthan', '/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/IPython/extensions', '/home/ec2-user/.ipython']


In [2]:
########################################
# 从s3同步数据
########################################


def sync_s3(file_name_list, s3_folder, local_folder):
    for f in file_name_list:
        print("file preparation: download src key {} to dst key {}".format(os.path.join(
            s3_folder, f), os.path.join(local_folder, f)))
        s3client.download_file(bucket, os.path.join(
            s3_folder, f), os.path.join(local_folder, f))


def write_to_s3(filename, bucket, key):
    print("upload s3://{}/{}".format(bucket, key))
    with open(filename, 'rb') as f:  # Read in binary mode
        # return s3client.upload_fileobj(f, bucket, key)
        return s3client.put_object(
            ACL='bucket-owner-full-control',
            Bucket=bucket,
            Key=key,
            Body=f
        )

def write_str_to_s3(content, bucket, key):
    print("write s3://{}/{}, content={}".format(bucket, key, content))
    s3client.put_object(Body=str(content).encode("utf8"), Bucket=bucket, Key=key, ACL='bucket-owner-full-control')

default_bucket = 'aws-gcr-rs-sol-demo-ap-southeast-1-522244679887'
default_prefix = 'sample-data'
parser = argparse.ArgumentParser()
parser.add_argument('--bucket', type=str)
parser.add_argument('--prefix', type=str)
args, _ = parser.parse_known_args()
bucket = args.bucket
prefix = args.prefix

print("bucket={}".format(bucket))
print("prefix='{}'".format(prefix))

out_s3_path = "s3://{}/{}/feature/content/inverted-list".format(bucket, prefix)

local_folder = 'info'
if not os.path.exists(local_folder):
    os.makedirs(local_folder)

file_name_list = ['complete_dkn_entity_embedding.npy','complete_dkn_word_embedding.npy']
s3_folder = '{}/model/rank/content/dkn_embedding_latest/'.format(prefix)
sync_s3(file_name_list, s3_folder, local_folder)
        
file_name_list = ['item.csv']
s3_folder = '{}/system/item-data'.format(prefix)
sync_s3(file_name_list, s3_folder, local_folder)

df_filter_item = pd.read_csv('info/item.csv',sep='_!_',names=['news_id','type_code','type','title','keywords','popularity','new'])

bucket=aws-gcr-rs-sol-demo-ap-southeast-1-522244679887
prefix='sample-data'
file preparation: download src key sample-data/model/rank/content/dkn_embedding_latest/complete_dkn_entity_embedding.npy to dst key info/complete_dkn_entity_embedding.npy
file preparation: download src key sample-data/model/rank/content/dkn_embedding_latest/complete_dkn_word_embedding.npy to dst key info/complete_dkn_word_embedding.npy
file preparation: download src key sample-data/system/item-data/item.csv to dst key info/item.csv


/home/ec2-user/anaconda3/envs/pytorch_p36/lib/python3.6/site-packages/ipykernel_launcher.py:55: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.


In [3]:
complete_dkn_word_embed = np.load("info/complete_dkn_word_embedding.npy")

In [4]:
# prepare model for batch process
meta_file_prefix = "{}/model/meta_files".format(prefix)
os.environ['GRAPH_BUCKET'] = bucket
os.environ['KG_DBPEDIA_KEY'] = '{}/kg_dbpedia.txt'.format(meta_file_prefix)
os.environ['KG_ENTITY_KEY'] = '{}/entities_dbpedia.dict'.format(meta_file_prefix)
os.environ['KG_RELATION_KEY'] = '{}/relations_dbpedia.dict'.format(meta_file_prefix)
os.environ['KG_DBPEDIA_TRAIN_KEY'] = '{}/kg_dbpedia_train.txt'.format(meta_file_prefix)
os.environ['KG_ENTITY_TRAIN_KEY'] = '{}/entities_dbpedia_train.dict'.format(meta_file_prefix)
os.environ['KG_RELATION_TRAIN_KEY'] = '{}/relations_dbpedia_train.dict'.format(meta_file_prefix)
os.environ['KG_ENTITY_INDUSTRY_KEY'] = '{}/entity_industry.txt'.format(meta_file_prefix)
os.environ['KG_VOCAB_KEY'] = '{}/vocab.json'.format(meta_file_prefix)
os.environ['DATA_INPUT_KEY'] = ''
os.environ['TRAIN_OUTPUT_KEY'] = '{}/'.format(meta_file_prefix)

kg_path = os.environ['GRAPH_BUCKET']
dbpedia_key = os.environ['KG_DBPEDIA_KEY']
entity_key = os.environ['KG_ENTITY_KEY']
relation_key = os.environ['KG_RELATION_KEY']
dbpedia_train_key = os.environ['KG_DBPEDIA_TRAIN_KEY']
entity_train_key = os.environ['KG_ENTITY_TRAIN_KEY']
relation_train_key = os.environ['KG_RELATION_TRAIN_KEY']
entity_industry_key = os.environ['KG_ENTITY_INDUSTRY_KEY']
vocab_key = os.environ['KG_VOCAB_KEY']
data_input_key = os.environ['DATA_INPUT_KEY']
train_output_key = os.environ['TRAIN_OUTPUT_KEY']

env = {
    'GRAPH_BUCKET': kg_path,
    'KG_DBPEDIA_KEY': dbpedia_key,
    'KG_ENTITY_KEY': entity_key,
    'KG_RELATION_KEY': relation_key,
    'KG_DBPEDIA_TRAIN_KEY': dbpedia_train_key,
    'KG_ENTITY_TRAIN_KEY': entity_train_key,
    'KG_RELATION_TRAIN_KEY': relation_train_key,
    'KG_ENTITY_INDUSTRY_KEY': entity_industry_key,
    'KG_VOCAB_KEY': vocab_key,
    'DATA_INPUT_KEY': data_input_key,
    'TRAIN_OUTPUT_KEY': train_output_key
}

print("Kg env: {}".format(env))
graph = kg.Kg(env)  # Where we keep the model when it's loaded
model = encoding.encoding(graph, env)

Kg env: {'GRAPH_BUCKET': 'aws-gcr-rs-sol-demo-ap-southeast-1-522244679887', 'KG_DBPEDIA_KEY': 'sample-data/model/meta_files/kg_dbpedia.txt', 'KG_ENTITY_KEY': 'sample-data/model/meta_files/entities_dbpedia.dict', 'KG_RELATION_KEY': 'sample-data/model/meta_files/relations_dbpedia.dict', 'KG_DBPEDIA_TRAIN_KEY': 'sample-data/model/meta_files/kg_dbpedia_train.txt', 'KG_ENTITY_TRAIN_KEY': 'sample-data/model/meta_files/entities_dbpedia_train.dict', 'KG_RELATION_TRAIN_KEY': 'sample-data/model/meta_files/relations_dbpedia_train.dict', 'KG_ENTITY_INDUSTRY_KEY': 'sample-data/model/meta_files/entity_industry.txt', 'KG_VOCAB_KEY': 'sample-data/model/meta_files/vocab.json', 'DATA_INPUT_KEY': '', 'TRAIN_OUTPUT_KEY': 'sample-data/model/meta_files/'}
load file: aws-gcr-rs-sol-demo-ap-southeast-1-522244679887/sample-data/model/meta_files/kg_dbpedia_train.txt
load file: aws-gcr-rs-sol-demo-ap-southeast-1-522244679887/sample-data/model/meta_files/kg_dbpedia.txt
load file: aws-gcr-rs-sol-demo-ap-southeast-

In [5]:
news_id_news_feature_dict = {}
map_words = {}
map_entities = {}

def analyze_map(raw_idx, map_dict, filter_idx):
    for idx in raw_idx:
        if idx == 0:
            filter_idx.append(0)
        else:
            if idx not in map_dict.keys():
                map_dict[idx] = len(map_dict)+1
            filter_idx.append(map_dict[idx])

for row in df_filter_item.iterrows():
    item_row = row[1]
    program_id = str(item_row['news_id'])
    title_result = model[item_row['title']]
    current_words = title_result[0]
    current_entities = title_result[1]
    filter_words = []
    filter_entities = []
    analyze_map(current_words, map_words, filter_words)
    analyze_map(current_entities, map_entities, filter_entities)
    # filter entities & filter words
    program_dict = {
        'entities': filter_entities,
        'words': filter_words
    }
    news_id_news_feature_dict[program_id] = program_dict

In [6]:
# clean data for graph train
path = '/home/ec2-user/workplace/recommender-system-solution/src/offline/news/item-feature-update-batch/aws-gcr-rs-sol-demo-ap-southeast-1-522244679887/sample-data/model/meta_files'
entities_dbpedia = os.path.join(path,'entities_dbpedia.dict')
relations_dbpedia = os.path.join(path,'relations_dbpedia.dict')
kg_dbpedia = os.path.join(path, 'kg_dbpedia.txt')
entities_dbpedia_train_path = os.path.join(path,'entities_dbpedia_train.dict')
relations_dbpedia_train_path = os.path.join(path,'relations_dbpedia_train.dict')
kg_dbpedia_train_path = os.path.join(path, 'kg_dbpedia_train.txt')
entities_dbpedia_f = pd.read_csv(entities_dbpedia, header=None, names=['e','e_name'])
relations_dbpedia_f = pd.read_csv(relations_dbpedia, header=None, names=['e','e_name']) 
kg_dbpedia_f = pd.read_csv(kg_dbpedia, delimiter='\t', header=None, names=['h','r','t'])

In [11]:
# map_entities -> train_entites
# constrcut from entites:
entities_dbpedia_slim = {}
relations_dbpedia_slim = {}

entities_dbpedia_train = {}
relations_dbpedia_train = {}

entities_dbpedia_train[0] = '0'
relations_dbpedia_train[0] = '0'

new_list_kg = []

def analyze_map_hrt(idx, map_dict, raw_content, train_dict):
    # 原始实体从0开始，所以需要归位进行寻找
    idx_test = idx - 1
    if idx_test not in map_dict.keys():
        map_dict[idx_test] = len(map_dict)+1
        filter_content = raw_content[raw_content.e == idx_test]
        train_dict[len(map_dict)] = filter_content['e_name'].values[0]
    return map_dict[idx_test]

for raw_entity, new_idx in map_entities.items():
    entity_id = raw_entity
    map_head_id = analyze_map_hrt(entity_id, entities_dbpedia_slim, entities_dbpedia_f, entities_dbpedia_train)
    
    kg_found_pd = kg_dbpedia_f[kg_dbpedia_f.h == entity_id]
#     print(kg_found_pd)
    for found_row in kg_found_pd.iterrows():
        relation_id = found_row[1]['r']
        tail_id = found_row[1]['t']
        map_relation_id = analyze_map_hrt(relation_id, relations_dbpedia_slim, relations_dbpedia_f, relations_dbpedia_train)
        map_tail_id = analyze_map_hrt(tail_id, entities_dbpedia_slim, entities_dbpedia_f, entities_dbpedia_train)
        # create new kg : h-r-t
        kg_row = {}
        kg_row['h'] = map_head_id
        kg_row['r'] = map_relation_id
        kg_row['t'] = map_tail_id        
        new_list_kg.append(kg_row)

kg_dbpedia_slim = pd.DataFrame(new_list_kg)

In [12]:
kg_dbpedia_slim.to_csv(kg_dbpedia_train_path, sep='\t', header=False, index=False)
import csv

with open(entities_dbpedia_train_path, 'w') as f:
    for key in entities_dbpedia_train.keys():
        f.write("%s,%s\n"%(key,entities_dbpedia_train[key]))
        
with open(relations_dbpedia_train_path, 'w') as f:
    for key in relations_dbpedia_train.keys():
        f.write("%s,%s\n"%(key,relations_dbpedia_train[key]))

In [14]:
# slim version
list_word_embedding = []
list_word_embedding.append([0]*300)
for raw_key, map_v in map_words.items(): 
    list_word_embedding.append(complete_dkn_word_embed[raw_key])

In [16]:
file_name = 'info/dkn_word_embedding.npy'
with open(file_name, "wb") as f:
    np.save(f, np.array(list_word_embedding))

write_to_s3(file_name,
            bucket,
            '{}/model/rank/content/dkn_embedding_latest/dkn_word_embedding.npy'.format(prefix))

write_to_s3(kg_dbpedia_train_path,
            bucket,
            '{}/kg_dbpedia_train.txt'.format(meta_file_prefix))

write_to_s3(entities_dbpedia_train_path,
            bucket,
            '{}/entities_dbpedia_train.dict'.format(meta_file_prefix))

write_to_s3(relations_dbpedia_train_path,
            bucket,
            '{}/relations_dbpedia_train.dict'.format(meta_file_prefix))

file_name = 'info/news_id_news_feature_dict.pickle'
out_file = open(file_name, 'wb')
pickle.dump(news_id_news_feature_dict, out_file)
out_file.close()
# s3_url = S3Uploader.upload(file_name, out_s3_path)
s3_url = write_to_s3(file_name, bucket, '{}/feature/content/inverted-list/news_id_news_feature_dict.pickle'.format(prefix))

upload s3://aws-gcr-rs-sol-demo-ap-southeast-1-522244679887/sample-data/model/rank/content/dkn_embedding_latest/dkn_word_embedding.npy
upload s3://aws-gcr-rs-sol-demo-ap-southeast-1-522244679887/sample-data/model/meta_files/kg_dbpedia_train.txt
upload s3://aws-gcr-rs-sol-demo-ap-southeast-1-522244679887/sample-data/model/meta_files/entities_dbpedia_train.dict
upload s3://aws-gcr-rs-sol-demo-ap-southeast-1-522244679887/sample-data/model/meta_files/relations_dbpedia_train.dict
upload s3://aws-gcr-rs-sol-demo-ap-southeast-1-522244679887/sample-data/feature/content/inverted-list/news_id_news_feature_dict.pickle


In [1]:
!aws s3 cp s3://gcr-rs-ops-ap-southeast-1-522244679887/news-open/model/rank/content/dkn_embedding_latest/dkn_word_embedding.npy s3://aws-gcr-rs-sol-demo-ap-southeast-1-522244679887/sample-data/model/rank/content/dkn_embedding_latest/complete_dkn_word_embedding.npy  --acl bucket-owner-full-control
!aws s3 cp s3://gcr-rs-ops-ap-southeast-1-522244679887/news-open/model/rank/content/dkn_embedding_latest/dkn_entity_embedding.npy s3://aws-gcr-rs-sol-demo-ap-southeast-1-522244679887/sample-data/model/rank/content/dkn_embedding_latest/complete_dkn_entity_embedding.npy  --acl bucket-owner-full-control

copy: s3://gcr-rs-ops-ap-southeast-1-522244679887/news-open/model/rank/content/dkn_embedding_latest/dkn_word_embedding.npy to s3://aws-gcr-rs-sol-demo-ap-southeast-1-522244679887/sample-data/model/rank/content/dkn_embedding_latest/complete_dkn_word_embedding.npy
copy: s3://gcr-rs-ops-ap-southeast-1-522244679887/news-open/model/rank/content/dkn_embedding_latest/dkn_entity_embedding.npy to s3://aws-gcr-rs-sol-demo-ap-southeast-1-522244679887/sample-data/model/rank/content/dkn_embedding_latest/complete_dkn_entity_embedding.npy


In [11]:
n = 0
for k, v in news_id_news_feature_dict.items():
    print("k {} v {}".format(k,v))
    if n > 10:
        break
    n = n + 1

k 6423135627981619458 v {'entities': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 2, 0, 0, 0], 'words': [1, 2, 0, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 0, 0, 0]}
k 6481778451320668430 v {'entities': [3, 0, 4, 0, 0, 0, 4, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'words': [13, 0, 14, 15, 16, 17, 14, 18, 19, 20, 21, 22, 23, 18, 24, 0]}
k 6487781016990646541 v {'entities': [0, 0, 5, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0, 0], 'words': [25, 26, 27, 28, 29, 30, 31, 32, 31, 33, 34, 35, 36, 37, 0, 38]}
k 6492610374146195725 v {'entities': [0, 0, 0, 0, 0, 0, 0, 0, 7, 0, 0, 0, 0, 0, 0, 0], 'words': [39, 40, 32, 17, 41, 42, 43, 17, 44, 45, 2, 40, 31, 32, 31, 0]}
k 6512940160806551816 v {'entities': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'words': [46, 47, 48, 49, 50, 51, 52, 53, 0, 0, 0, 0, 0, 0, 0, 0]}
k 6520157116831891716 v {'entities': [0, 0, 0, 0, 0, 0, 0, 8, 0, 0, 0, 0, 0, 0, 0, 0], 'words': [54, 55, 56, 0, 57, 26, 58, 59, 0, 0, 0, 0, 0, 0, 0, 0]}
k 6520548609547567367 v {'entities': [0, 0, 0, 0, 0, 0, 0, 0, 0